In [2]:
import pyfits 
import pandas as pd
from astropy.io import fits
import numpy as np
from astropy.table import Table, join
import math
import astropy.units as u

Reads in the data from SME and The Cannon. Combines them based on sobject_id

In [3]:
sme_abund=Table.read('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/sobject_iraf_sme_1.1_k2.fits', format='fits')

In [4]:
cannon_param=Table.read('/Users/kschles/Documents/GALAH/wg4output/wg4_04292016/sobject_iraf_cannon_1.1_k2.fits', format='fits')

In [5]:
wg4_combined=join(sme_abund, cannon_param[['sobject_id', 'teff_cannon', 'e_teff_cannon', 'logg_cannon', 'e_logg_cannon',\
                                   'feh_cannon','e_feh_cannon','alpha_fe_cannon','e_alpha_fe_cannon',\
                                   'chi2_cannon','flag_cannon','vers_cannon','rv_cannon','e_rv_cannon',\
                                   'ebv_cannon']], keys=['sobject_id'], join_type='right')

Want to merge with information from IRAF output table to get GUESS info, etc. 

In [6]:
iraf_data=pd.read_csv("/Users/kschles/Documents/GALAH/iraf_v5.1/iraf_dr51_new_05042016.csv")
## The GUESS values are often set to 999. or 9999. rather than null. So I need to adjust this. 
iraf_data.loc[np.where(iraf_data['rv_c2_guess'].astype(float)==999.)[0],'rv_c2_guess']=np.nan

In [6]:
iraf_data.columns.values

array(['sobject_id', 'galah_id', 'cob_id', 'pivot', 'field_id', 'ra',
       'dec', 'ebv', 'barycentric', 'rv_c1_guess', 'rv_c2_guess',
       'rv_c3_guess', 'rv_c4_guess', 'rv_guess', 'e_rv_guess',
       'teff_guess', 'logg_guess', 'feh_guess', 'flag_guess',
       'snr_c1_guess', 'snr_c2_guess', 'snr_c3_guess', 'snr_c4_guess',
       'snr_c1_iraf', 'snr_c2_iraf', 'snr_c3_iraf', 'snr_c4_iraf',
       'snr2_c1_iraf', 'snr2_c2_iraf', 'snr2_c3_iraf', 'snr2_c4_iraf',
       'red_flag', 'red_vers'], dtype=object)

In [7]:
iraf_data.rename(columns={'teff' : 'teff_guess', 'logg': 'logg_guess', \
                            'feh' : 'feh_guess', 'vmean' : 'rv_guess', 'vstd' : 'e_rv_guess'}, inplace=True)
iraf_data['sobject_id']=iraf_data['sobject_id'].astype(int)

In [8]:
iraf_table=Table.from_pandas(iraf_data)

In [9]:
temp_output=join(wg4_combined, iraf_table[['sobject_id', 'field_id', 'teff_guess', 'logg_guess',\
                                           'feh_guess', 'rv_guess', 'e_rv_guess', 'flag_guess']], keys=['sobject_id'], join_type='left')

In [10]:
temp_output['vers_guess']='1.1'

In [16]:
temp_output['rv_guess'].unit='km/s'
temp_output['e_rv_guess'].unit='km/s'
temp_output['teff_guess'].unit='K'
#temp_output['logg_guess'].unit=u.Unit("dex(cm/s2)")
temp_output['feh_guess'].unit='dex'

In [12]:
# Set column order 
## Remove modes Ru, Mo, Sm, Sr from the results table
new_order=['sobject_id','galah_id','ra','dec','field_id', 'red_pipeline','red_flags','teff_cannon','e_teff_cannon','logg_cannon',\
           'e_logg_cannon','feh_cannon','e_feh_cannon','alpha_fe_cannon','e_alpha_fe_cannon','chi2_cannon','flag_cannon',\
           'vers_cannon','teff_sme','e_teff_sme','logg_sme','e_logg_sme','feh_sme','e_feh_sme','alpha_fe_sme',\
           'e_alpha_fe_sme','flag_sme','vers_sme', 'teff_guess', 'logg_guess', 'feh_guess',\
           'flag_guess', 'vers_guess', 'rv_guess', 'e_rv_guess', \
           'rv_cannon','e_rv_cannon','rv_sme','e_rv_sme','vsini_sme','e_vsini_sme',\
           'Fe_abund_sme','Fe_e_abund_sme','Fe_aflag_sme','Fe_chi_sme',\
           'Li_abund_sme','Li_e_abund_sme','Li_aflag_sme','Li_chi_sme',\
           'C_abund_sme','C_e_abund_sme','C_aflag_sme','C_chi_sme',\
           'O_abund_sme','O_e_abund_sme','O_aflag_sme','O_chi_sme',\
           'Na_abund_sme','Na_e_abund_sme','Na_aflag_sme','Na_chi_sme',\
           'Mg_abund_sme','Mg_e_abund_sme','Mg_aflag_sme','Mg_chi_sme',\
           'Al_abund_sme','Al_e_abund_sme','Al_aflag_sme','Al_chi_sme',\
           'Si_abund_sme','Si_e_abund_sme','Si_aflag_sme','Si_chi_sme',\
           'K_abund_sme','K_e_abund_sme','K_aflag_sme','K_chi_sme',\
           'Ca_abund_sme','Ca_e_abund_sme','Ca_aflag_sme','Ca_chi_sme',\
           'Sc_abund_sme','Sc_e_abund_sme','Sc_aflag_sme','Sc_chi_sme',\
           'Ti_abund_sme','Ti_e_abund_sme','Ti_aflag_sme','Ti_chi_sme',\
           'V_abund_sme','V_e_abund_sme','V_aflag_sme','V_chi_sme',\
           'Cr_abund_sme','Cr_e_abund_sme','Cr_aflag_sme','Cr_chi_sme',\
           'Mn_abund_sme','Mn_e_abund_sme','Mn_aflag_sme','Mn_chi_sme',\
           'Co_abund_sme','Co_e_abund_sme','Co_aflag_sme','Co_chi_sme',\
           'Ni_abund_sme','Ni_e_abund_sme','Ni_aflag_sme','Ni_chi_sme',\
           'Cu_abund_sme','Cu_e_abund_sme','Cu_aflag_sme','Cu_chi_sme',\
           'Zn_abund_sme','Zn_e_abund_sme','Zn_aflag_sme','Zn_chi_sme',\
           'Y_abund_sme','Y_e_abund_sme','Y_aflag_sme','Y_chi_sme',\
           'Ba_abund_sme','Ba_e_abund_sme','Ba_aflag_sme','Ba_chi_sme',\
           'La_abund_sme','La_e_abund_sme','La_aflag_sme','La_chi_sme',\
           'Nd_abund_sme','Nd_e_abund_sme','Nd_aflag_sme','Nd_chi_sme',\
           'Eu_abund_sme','Eu_e_abund_sme','Eu_aflag_sme','Eu_chi_sme',\
           'Rb_abund_sme','Rb_e_abund_sme','Rb_aflag_sme','Rb_chi_sme']


In [13]:
# All to lower case
new_order=[x.lower() for x in new_order]


In [14]:
output=temp_output[new_order]

In [18]:
#output.write('wg4_04292016/test_filename.csv', format='csv')
output.write('wg4_04292016/sobject_iraf_k2.fits', format='fits')